In [31]:
import torch
import pandas as pd
import pickle
import numpy as np

In [32]:
train = pd.read_pickle("../data/ml-20m-split/train.pkl")


In [33]:
freqs = train.groupby("user").size().reset_index().rename(columns={0: "num_items"})


In [34]:
freqs

,user,num_items
0,0,175
1,1,61
2,2,187
3,3,28
4,4,66
...,...,...
138488,138488,38
138489,138489,151
138490,138490,22
138491,138491,82


In [35]:
num_items_col = []
prev_user = 0
prev_num_items = freqs.iloc[prev_user].num_items
for row in train.itertuples(index=False):
  if row.user != prev_user:
    prev_user = row.user
    prev_num_items = freqs.iloc[prev_user].num_items
    if prev_user % 10000 == 0:
      print(prev_user)
  num_items_col.append(prev_num_items)
  

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000


In [36]:
len(num_items_col)

13999448

In [37]:
train["user_num_items"] = num_items_col

In [38]:
group = train.groupby("user_num_items")

In [39]:
train

,user,item,item_id,rating,timestamp,user_num_items
0,0,1,2,3.5,1112486027,175
1,0,28,29,3.5,1112484676,175
3,0,46,47,3.5,1112484727,175
4,0,49,50,3.5,1112484580,175
5,0,110,112,3.5,1094785740,175
...,...,...,...,...,...,...
20000256,138492,13485,66762,4.5,1255805408,373
20000257,138492,13664,68319,4.5,1260209720,373
20000259,138492,13862,69526,4.5,1259865108,373
20000260,138492,13875,69644,3.0,1260209457,373


In [40]:
i = 0
BATCH_SIZE = 32
batches = []
for (user_num_items, block) in group:
  i += 1
  #print(row.sample(frac=1))
  for (idx, batch) in block.sample(frac=1).groupby(np.arange(block.shape[0]) // BATCH_SIZE):
    batches.append(batch)
  if i % 10 == 0:
    print(i)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890


In [ ]:
row0 = group.nth(0)

In [ ]:
row0

In [ ]:
for (i, r) in therow.groupby(np.arange(therow.shape[0]) // 32):
  print(r)
  if i == 5:
    break

In [56]:
len(batches)

438151

In [57]:
with open("../data/ml-20m-split/train_batched.pkl", "wb") as f:
  pickle.dump(batches, f)

In [28]:
arr = np.array(batches[52342].drop(["item_id", "timestamp", "user_num_items"], axis=1))

In [30]:
arr.dtype

dtype('float64')

In [22]:
batches2 = np.stack([np.array(batches[i].drop(["item_id", "timestamp", "user_num_items"], axis=1)) for i in range(len(batches))])

KeyboardInterrupt: 

In [23]:
to_stack = []
for i in range(len(batches)):
  if i % 10000 == 0:
    print(i)
  to_stack.append(np.array(batches[i].drop(["item_id", "timestamp", "user_num_items"], axis=1)))

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000


In [24]:
batches2 = np.stack(to_stack)

ValueError: all input arrays must have the same shape

In [27]:
np.save("../data/ml-20m-split/train_batched_numpy.npy", to_stack)